# Demo of CCC for PSL Meetup

This notebook provides a demonstration of the Cost-of-Capital-Calculator (CCC) for the PSL Meetup on April 29, 2019.

To run this notebook on your machine, you will need to follow the instructions to install CCC as described in the CCC README [here](https://github.com/PSLmodels/Cost-of-Capital-Calculator).  In particular, you need to:

* Install the [Anaconda distribution](https://www.anaconda.com/distribution/) of Python
* Install the CCC package by typing `conda install -c conda-forge ccc` (or `pip install cost-of-captial-calculator`) in the command prompt.

Once you follow the above, you will be ready to work with this Jupyter Notebook.

## First things first, import necessary packages

In [4]:
# To install ccc package (if not already):
import sys
if 'ccc' not in sys.modules:
    !pip install cost-of-capital-calculator

In [5]:
# import packages
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
# import CCC classes that we'll work with
from ccc.data import Assets
from ccc.parameters import Specification, DepreciationParams
from ccc.calculator import Calculator
# to print bokeh plots inline
output_notebook()

Loading BokehJS ...

## Create an instance of the Assets class

This is the class that contains the data that underlie CCC.  The basic object is a Pandas DataFrame where each row represents a combination of a specific type of asset, industry (approximately 6-digit NAICS) and tax treatment.  The columns represent names and codes for the asset and industry classifications, the tax depreciation rules used for that asset, and the rate of economic depreciation (`delta`).

In [6]:
assets = Assets()
assets.df.head(n=5)

,Unnamed: 0,tax_treat,assets,bea_asset_code,bea_ind_code,Industry,minor_code_alt,major_asset_group,minor_asset_group,major_industry,asset_name,delta
0,0,corporate,0.0,RD32,110C,Farms,111,Intellectual Property,Intellectual Property,"Agriculture, forestry, fishing, and hunting",Aerospace products and parts manufacturing,0.22
1,1,corporate,0.0,RD32,113F,"Forestry, fishing, and related activities",113,Intellectual Property,Intellectual Property,"Agriculture, forestry, fishing, and hunting",Aerospace products and parts manufacturing,0.22
2,2,corporate,0.0,RD32,113F,"Forestry, fishing, and related activities",114,Intellectual Property,Intellectual Property,"Agriculture, forestry, fishing, and hunting",Aerospace products and parts manufacturing,0.22
3,3,corporate,0.0,RD32,2110,Oil and gas extraction,211110,Intellectual Property,Intellectual Property,Mining,Aerospace products and parts manufacturing,0.22
4,4,corporate,0.0,RD32,2120,"Mining, except oil and gas",212110,Intellectual Property,Intellectual Property,Mining,Aerospace products and parts manufacturing,0.22


## Create instances of the two parameters classes

The `Specification` class contains many of the model parameters, although depreciation system parameters are contained in the `DepreciationParams` class.  Both are required arguments for the `Calculator` object.

A `Specification` object has methods that load the data from a file that contains the default parameter values (`default_parameters.json`) and then stores them as attributes of the `Specification` class object.  

`p.u` looks into the instance of this class named `p` and executing the cell below will show you that `p.u` is dictionary containtin marginal tax rates on corporate and pass-through income.

Note that the `Specification` class has some arguments, such as `call_tc`, which defaults to `False`, but if set to `True` will call the Tax-Calculator to estimate marginal tax rates on individual filers' income.

Similar to `Specification`, `DepreciationParams` loads default parameters from a JSON file, in this case the file `tax_depreciation_rules.json`.

In [7]:
# Create an instance of the Specification class
# p = Specification()
p = Specification(call_tc=True)
# Look at attributes
p.u

Calculator initial year =  2014
Calculator year =  2024
year:  2024
{'tau_pt': array([0.20147244]), 'tau_div': array([0.1723512]), 'tau_int': array([0.31217545]), 'tau_scg': array([0.28524757]), 'tau_lcg': array([0.184897]), 'tau_td': array([0.204116]), 'tau_h': array([0.03463544])}


{'c': array([0.21]), 'pt': array([0.20147244])}

In [8]:
# Create an instance of the DepreciationParams class
dp = DepreciationParams()
# Look at an attribute of the dp object
dp.asset[0]

OrderedDict([('value', {'life': 5.0, 'method': 'DB 200%'}),
             ('asset_name', 'Mainframes'),
             ('BEA_code', 'EP1A'),
             ('minor_asset_group', 'Computers and Software'),
             ('major_asset_group', 'Equipment'),
             ('ADS_life', 5.0),
             ('GDS_life', 5.0),
             ('system', 'GDS'),
             ('year', 2020)])

## Create an instance of the Calculator class

This class does the calculations on the data.  It takes as arguments the data object (named `assets` here) and the parameters object (named `p` here).

Excuting the cell below creates an instance of the calculator class with these data and parameters, but does not yet excecute any calculations.

In [9]:
# Create an instance of the Calculator class
calc1 = Calculator(p, dp, assets)

With an instance of the `Calculator` class created, we can start doing some calculations with these data.

For instance, we can compute a table of the share of corporate vs non-corporate assets across each industry.    

In [10]:
# Look at shares of assets across industry
calc1.asset_share_table()

,Industry,Corporate,Pass-Through
0,"Agriculture, forestry, fishing, and hunting",0.309688,0.690312
1,Mining,0.826839,0.173161
2,Utilities,0.948540,0.051460
3,Construction,0.286840,0.713160
4,Manufacturing,0.859014,0.140986
5,Wholesale trade,0.715166,0.284834
6,Retail trade,0.728628,0.271372
7,Transportation and warehousing,0.789652,0.210348
8,Information,0.918722,0.081278
9,Finance and insurance,0.786056,0.213944


## Specifying a reform policy

To see some more interesting results, we will want to create another `Calculator` object with a change in policy or economic assumptions.  We can do this in a way analogous to our original instantiation of the `Calculator` object above.  In particular, we'll need to create a new `Specification` object (we can use the same underlying data, which was in the object we named `assets`).

In the code below, we'll specify our "reform" as current law tax policy for 2026 (the baseline parameters above came from the default model year, 2019).  In addition, we'll increase the corporate income tax rate from 21 to 25%.

In [11]:
# Create another policy
p2 = Specification(year=2026)
p2.update_specification({'CIT_rate': 0.35})
calc2 = Calculator(p2, dp, assets)

## Tabular output

Now with two `Calculator` objects named `calc1` and `calc2` (representing the baseline and reform policies), we are ready to compute some of the changes in effective tax rates, cost of capital, or other variables measured in this model.

We start with an overall summary table showing the marginal effective total tax rates (METTRs) for all investments, corporate investments, and pass-through investments under varying financing assumptions.  This is done through the `summary_table` function.  It takes a calculator object as an argument.

In [12]:
# Look at differences in METTRs between the two policies
calc1.summary_table(calc2) # calc1 is the baseline, calc2 the reform

,,Marginal Effective Total Tax Rate Under Baseline Policy,Marginal Effective Total Tax Rate Under Reform Policy,Change from Baseline (pp)
0,Overall,18.563352,24.710063,6.146711
1,Corporations,19.197626,27.536312,8.338686
2,Equity Financed,22.052902,34.270098,12.217196
3,Debt Financed,8.933048,-5.186307,-14.119355
4,Pass-Through Entities,19.312399,22.497415,3.185016
5,Equity Financed,17.629037,22.436954,4.807917
6,Debt Financed,25.652332,22.875706,-2.776626


the `Calculator.summary_table()` method defaults to showing the results for the METTR, but there is a keyword argument that would allow you to view the output for other variables computed in CCC.  We can use this to see changes in the cost of capital (denoted by $\rho$ in the model):

In [13]:
# Look at how the cost of capital changed
calc1.summary_table(calc2, output_variable='rho')

,,Cost of Capital Under Baseline Policy,Cost of Capital Under Reform Policy,Change from Baseline (pp)
0,Overall,5.777634,6.249323,0.471689
1,Corporations,5.822987,6.493061,0.670074
2,Equity Financed,6.719745,7.968742,1.248997
3,Debt Financed,3.923541,3.396877,-0.526664
4,Pass-Through Entities,5.723574,5.958787,0.235214
5,Equity Financed,6.358851,6.753018,0.394167
6,Debt Financed,4.171191,4.021020,-0.150171


One can also save results to disk by specifying an output type ('excel', 'json', 'csv', 'tex') and a file path: 

In [14]:
# Save these results to disk
calc1.summary_table(calc2, output_variable='rho', output_type='excel', path='cc_table.xlsx')

DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).

There are also `Calculator` methods to compute summary tables by asset type or industry.  These are computed in the next two cells.

In [ ]:
# Summary by asset type
calc1.asset_summary_table(calc2)

deprec_df                                 value  asset_name BEA_code  \
0  {'life': 5.0, 'method': 'DB 200%'}  Mainframes     EP1A   
1  {'life': 5.0, 'method': 'DB 200%'}         PCs     EP1B   
2  {'life': 5.0, 'method': 'DB 200%'}       DASDs     EP1C   
3  {'life': 5.0, 'method': 'DB 200%'}    Printers     EP1D   
4  {'life': 5.0, 'method': 'DB 200%'}   Terminals     EP1E   

        minor_asset_group major_asset_group  ADS_life  GDS_life system  year  
0  Computers and Software         Equipment       5.0       5.0    GDS  2020  
1  Computers and Software         Equipment       5.0       5.0    GDS  2020  
2  Computers and Software         Equipment       5.0       5.0    GDS  2020  
3  Computers and Software         Equipment       5.0       5.0    GDS  2020  
4  Computers and Software         Equipment       5.0       5.0    GDS  2020  
deprec_df 2    asset_name BEA_code       minor_asset_group major_asset_group  ADS_life  \
0  Mainframes     EP1A  Computers and Software       

,Category,Marginal Effective Total Tax Rate Under Baseline Policy,Marginal Effective Total Tax Rate Under Reform Policy,Change from Baseline (pp)
0,Overall,17.639885,25.524466,7.884581
1,Corporate,17.755445,28.320144,10.564699
2,Equipment,6.748557,22.505995,15.757439
3,Structures,18.403895,27.871517,9.467623
4,Intellectual Property,6.748557,11.267132,4.518575
5,Inventories,29.224527,39.958412,10.733886
6,Land,26.331360,35.884178,9.552818
7,Pass-through,19.007441,23.318798,4.311357
8,Equipment,1.961381,15.050433,13.089052
9,Structures,17.622599,21.473278,3.850679


In [ ]:
# Summary by industry
calc1.industry_summary_table(calc2)

deprec_df                                 value  asset_name BEA_code  \
0  {'life': 5.0, 'method': 'DB 200%'}  Mainframes     EP1A   
1  {'life': 5.0, 'method': 'DB 200%'}         PCs     EP1B   
2  {'life': 5.0, 'method': 'DB 200%'}       DASDs     EP1C   
3  {'life': 5.0, 'method': 'DB 200%'}    Printers     EP1D   
4  {'life': 5.0, 'method': 'DB 200%'}   Terminals     EP1E   

        minor_asset_group major_asset_group  ADS_life  GDS_life system  year  
0  Computers and Software         Equipment       5.0       5.0    GDS  2020  
1  Computers and Software         Equipment       5.0       5.0    GDS  2020  
2  Computers and Software         Equipment       5.0       5.0    GDS  2020  
3  Computers and Software         Equipment       5.0       5.0    GDS  2020  
4  Computers and Software         Equipment       5.0       5.0    GDS  2020  
deprec_df 2    asset_name BEA_code       minor_asset_group major_asset_group  ADS_life  \
0  Mainframes     EP1A  Computers and Software       

,Category,Marginal Effective Total Tax Rate Under Baseline Policy,Marginal Effective Total Tax Rate Under Reform Policy,Change from Baseline (pp)
0,Overall,17.639885,25.524466,7.884581
1,Corporate,17.755445,28.320144,10.564699
2,"Agriculture, forestry, fishing, and hunting",18.621881,30.896046,12.274165
3,Mining,9.714456,17.971900,8.257444
4,Utilities,7.770042,20.115997,12.345955
5,Construction,19.159651,30.624693,11.465042
6,Manufacturing,18.050971,27.120119,9.069148
7,Wholesale trade,23.492232,34.533774,11.041542
8,Retail trade,24.479352,34.948319,10.468967
9,Transportation and warehousing,10.992693,20.708297,9.715604


## Visualizations

If one wants to visualize the effects of changes in tax policy, the `Calculator` class has a few methods for this.  

We can use the `grouped_bar` method to show differential effects across assets (the default):

In [ ]:
# Visualizing changes by asset type
aplot = calc1.grouped_bar(calc2)
show(aplot)

deprec_df                                 value  asset_name BEA_code  \
0  {'life': 5.0, 'method': 'DB 200%'}  Mainframes     EP1A   
1  {'life': 5.0, 'method': 'DB 200%'}         PCs     EP1B   
2  {'life': 5.0, 'method': 'DB 200%'}       DASDs     EP1C   
3  {'life': 5.0, 'method': 'DB 200%'}    Printers     EP1D   
4  {'life': 5.0, 'method': 'DB 200%'}   Terminals     EP1E   

        minor_asset_group major_asset_group  ADS_life  GDS_life system  year  
0  Computers and Software         Equipment       5.0       5.0    GDS  2020  
1  Computers and Software         Equipment       5.0       5.0    GDS  2020  
2  Computers and Software         Equipment       5.0       5.0    GDS  2020  
3  Computers and Software         Equipment       5.0       5.0    GDS  2020  
4  Computers and Software         Equipment       5.0       5.0    GDS  2020  
deprec_df 2    asset_name BEA_code       minor_asset_group major_asset_group  ADS_life  \
0  Mainframes     EP1A  Computers and Software       

or we can change from the default asset category split to a split by industry by changing the value of the `group_by_asset` keyword argument:

In [ ]:
# Visualizing changes by industry
iplot = calc1.grouped_bar(calc2, group_by_asset=False)
show(iplot)

deprec_df                                 value  asset_name BEA_code  \
0  {'life': 5.0, 'method': 'DB 200%'}  Mainframes     EP1A   
1  {'life': 5.0, 'method': 'DB 200%'}         PCs     EP1B   
2  {'life': 5.0, 'method': 'DB 200%'}       DASDs     EP1C   
3  {'life': 5.0, 'method': 'DB 200%'}    Printers     EP1D   
4  {'life': 5.0, 'method': 'DB 200%'}   Terminals     EP1E   

        minor_asset_group major_asset_group  ADS_life  GDS_life system  year  
0  Computers and Software         Equipment       5.0       5.0    GDS  2020  
1  Computers and Software         Equipment       5.0       5.0    GDS  2020  
2  Computers and Software         Equipment       5.0       5.0    GDS  2020  
3  Computers and Software         Equipment       5.0       5.0    GDS  2020  
4  Computers and Software         Equipment       5.0       5.0    GDS  2020  
deprec_df 2    asset_name BEA_code       minor_asset_group major_asset_group  ADS_life  \
0  Mainframes     EP1A  Computers and Software       

There's also a plot that illustrates the range of the effects of taxes on investments across asset types, by showing the min, max, and mean values:

In [ ]:
# Plot to show variation in METTRs across assets
rplot = calc1.range_plot(calc2, output_variable='metr')
show(rplot)

deprec_df                                 value  asset_name BEA_code  \
0  {'life': 5.0, 'method': 'DB 200%'}  Mainframes     EP1A   
1  {'life': 5.0, 'method': 'DB 200%'}         PCs     EP1B   
2  {'life': 5.0, 'method': 'DB 200%'}       DASDs     EP1C   
3  {'life': 5.0, 'method': 'DB 200%'}    Printers     EP1D   
4  {'life': 5.0, 'method': 'DB 200%'}   Terminals     EP1E   

        minor_asset_group major_asset_group  ADS_life  GDS_life system  year  
0  Computers and Software         Equipment       5.0       5.0    GDS  2020  
1  Computers and Software         Equipment       5.0       5.0    GDS  2020  
2  Computers and Software         Equipment       5.0       5.0    GDS  2020  
3  Computers and Software         Equipment       5.0       5.0    GDS  2020  
4  Computers and Software         Equipment       5.0       5.0    GDS  2020  
deprec_df 2    asset_name BEA_code       minor_asset_group major_asset_group  ADS_life  \
0  Mainframes     EP1A  Computers and Software       

changing to show the effects on pass-through businesses and for a different output variable:

In [ ]:
show(rplot)

In [ ]:
# Plot to show variation in METTRs across assets
output_notebook()
rplot2 = calc1.range_plot(calc2, corporate=False, output_variable='mettr')
show(rplot2)

Loading BokehJS ...

deprec_df                                 value  asset_name BEA_code  \
0  {'life': 5.0, 'method': 'DB 200%'}  Mainframes     EP1A   
1  {'life': 5.0, 'method': 'DB 200%'}         PCs     EP1B   
2  {'life': 5.0, 'method': 'DB 200%'}       DASDs     EP1C   
3  {'life': 5.0, 'method': 'DB 200%'}    Printers     EP1D   
4  {'life': 5.0, 'method': 'DB 200%'}   Terminals     EP1E   

        minor_asset_group major_asset_group  ADS_life  GDS_life system  year  
0  Computers and Software         Equipment       5.0       5.0    GDS  2020  
1  Computers and Software         Equipment       5.0       5.0    GDS  2020  
2  Computers and Software         Equipment       5.0       5.0    GDS  2020  
3  Computers and Software         Equipment       5.0       5.0    GDS  2020  
4  Computers and Software         Equipment       5.0       5.0    GDS  2020  
deprec_df 2    asset_name BEA_code       minor_asset_group major_asset_group  ADS_life  \
0  Mainframes     EP1A  Computers and Software       

There's a bubble plot too, though it doesn't yet allow for many options (currently only plots METTRs for corporate entities):

In [ ]:
bplot = calc1.asset_bubble(calc2)
show(bplot)

deprec_df                                 value  asset_name BEA_code  \
0  {'life': 5.0, 'method': 'DB 200%'}  Mainframes     EP1A   
1  {'life': 5.0, 'method': 'DB 200%'}         PCs     EP1B   
2  {'life': 5.0, 'method': 'DB 200%'}       DASDs     EP1C   
3  {'life': 5.0, 'method': 'DB 200%'}    Printers     EP1D   
4  {'life': 5.0, 'method': 'DB 200%'}   Terminals     EP1E   

        minor_asset_group major_asset_group  ADS_life  GDS_life system  year  
0  Computers and Software         Equipment       5.0       5.0    GDS  2020  
1  Computers and Software         Equipment       5.0       5.0    GDS  2020  
2  Computers and Software         Equipment       5.0       5.0    GDS  2020  
3  Computers and Software         Equipment       5.0       5.0    GDS  2020  
4  Computers and Software         Equipment       5.0       5.0    GDS  2020  
deprec_df 2    asset_name BEA_code       minor_asset_group major_asset_group  ADS_life  \
0  Mainframes     EP1A  Computers and Software       

## Summary

This notebook provides a brief example of how one would work with CCC.  Please explore the source code to see additional flexiblity in the functions.  And please leave any questions or suggestions in the CCC repo at [https://github.com/PSLmodels/Cost-of-Capital-Calculator/issues](https://github.com/PSLmodels/Cost-of-Capital-Calculator/issues).